In [10]:
import pandas as pd
import requests
import io

In [11]:
def load_dataset(tipo):
    if tipo == "Confirmed":
        file = "time_series_covid19_confirmed_global.csv"
    elif tipo == "Deaths":
        file = "time_series_covid19_deaths_global.csv"
    else:
        file = "time_series_covid19_recovered_global.csv"

    url_confirmed=f"https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/{file}"
    s=requests.get(url_confirmed).content
    df=pd.read_csv(io.StringIO(s.decode('utf-8')))
    df = df.drop(["Province/State", "Lat", "Long"], axis=1)
    df = pd.DataFrame(df.set_index("Country/Region").stack()).reset_index()
    df = df.rename(columns={"Country/Region":"Country","level_1":"Date", 0:tipo})
    df["Date"] = pd.to_datetime(df["Date"])
    df = df.groupby(["Date", "Country"])[tipo].max().reset_index()
    return df

In [15]:
confirmed_df = load_dataset("Confirmed")
deaths_df = load_dataset("Deaths")
recovered_df = load_dataset("Recovered")

confirmed_df.to_csv("Confirmed.csv", index=False)
deaths_df.to_csv("Deaths.csv", index=False)
recovered_df.to_csv("Recovered.csv", index=False)